In [1]:
from rmgpy import settings
from rmgpy.data.rmg import RMGDatabase
from rmgpy.molecule import Molecule, Atom, Bond
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from copy import deepcopy
import itertools
import numpy as np
import yaml

In [2]:
database = RMGDatabase()
database.load(
    path = settings['database.directory'],
    thermo_libraries = ['surfaceThermoPt111', 'primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC','DFT_QCI_thermo'],  # Can add others if necessary
    kinetics_families = ['surface','default','surface_development'],
    reaction_libraries = [],
    kinetics_depositories = ['training'],
)

In [3]:
def get_labeled_rxns(reactants,products):
    reacts = [m.copy(deep=True) for m in reactants]
    if products:
        prods = [m.copy(deep=True) for m in products]
    else:
        prods = None
    rxns = []
    for family in database.kinetics.families.values():
        if "Surface" not in family.label:
            continue
        #skip barrierless surface families
        elif family.label in ["Surface_Adsorption_vdW","Surface_Adsorption_Single","Surface_Adsorption_Double","Surface_vdW_to_Bidentate","Surface_Adsorption_Bidentate"]: 
            continue
        elif family.label == "Surface_Migration" and len(reactants) == 1 and len(reactants[0].get_surface_sites()) > 1:
            continue
        for x in reacts: #at start of each family clean up any leftover labels on the reactants from generate_reactions
            x.clear_labeled_atoms()
        try:
            newrxns = family.generate_reactions(reacts,products=prods,delete_labels=False,relabel_atoms=False)
        except Exception as e:
            print(family)
            for r in reactants:
                print(r.to_adjacency_list())
            raise e
        
        if len(newrxns) == 0:
            continue
        else:
            root = None
            for r in family.forward_template.reactants:
                if root:
                    root = root.merge(r.item)
                else:
                    root = r.item.copy(deep=True)
                    
        for newrxn in newrxns:
            molr = None
            for react in newrxn.reactants:
                if molr:
                    molr = molr.merge(react)
                else:
                    molr = deepcopy(react)
            molr.update()
            
            forward = molr.is_subgraph_isomorphic(root,save_order=True,generate_initial_map=True)
            try:
                prod = family.apply_recipe(newrxn.reactants, forward=forward, unique=True, relabel_atoms=False)
                newrxn.products = prod
            except Exception as e:
                print("apply failed")
                print(str(newrxn))
                print("in forward")
                print(len(newrxns))
                print(family)
                print(forward)
                print(newrxn.is_forward)
                print("root")
                print(root.to_adjacency_list())
                print("molr")
                print(molr.to_adjacency_list())
                print("reactants")
                for m in reacts:
                    print(m.to_adjacency_list())
                for m in newrxn.reactants:
                    print(m.to_adjacency_list())
                for m in newrxn.products:
                    print(m.to_adjacency_list())
                raise e
        
        #check label mapping
        end = False
        for r in newrxns:
            labeled_atoms_react = dict()
            for m in r.reactants:
                labeled_atoms_react.update(m.get_all_labeled_atoms())
            labeled_atoms_prod = dict()
            for m in r.products:
                labeled_atoms_prod.update(m.get_all_labeled_atoms())
            
            for k,v in labeled_atoms_react.items():
                if v.symbol != labeled_atoms_prod[k].symbol:
                    print("errored")
                    print(len(newrxns))
                    print(family)
                    print(r.is_forward)
                    for m in reactants:
                        print(m.to_adjacency_list())
                    for m in r.reactants:
                        print(m.to_adjacency_list())
                    for m in r.products:
                        print(m.to_adjacency_list())
                    print("apply recipe")
                    prod = family.apply_recipe(r.reactants, forward=True, unique=True, relabel_atoms=False)
                    for m in prod:
                        print(m.to_adjacency_list())
                    end = True
        if end:
            raise ValueError
                    
        outrxns = []
        for r in newrxns:
            for outr in outrxns:
                if outr.is_isomorphic(r,either_direction=True,generate_initial_map=True,strict=False):
                    break
            else:
                outrxns.append(r)
        if outrxns:
            rxns.extend([x.copy() for x in outrxns]) #when done with the family we deep copy reactions so modifying reactants won't affect things
        
    return rxns

def labeled_rxn_to_dict(rxn):
    considered_families = ['Surface_Abstraction', 'Surface_Abstraction_Beta',
       'Surface_Abstraction_Beta_double_vdW',
       'Surface_Abstraction_Beta_vdW', 'Surface_Abstraction_Single_vdW',
       'Surface_Abstraction_vdW', 'Surface_Adsorption_Bidentate',
       'Surface_Adsorption_Dissociative',
       'Surface_Adsorption_Dissociative_Double',
       'Surface_Adsorption_Single', 'Surface_Bidentate_Dissociation',
       'Surface_Dissociation', 'Surface_Dissociation_Beta',
       'Surface_Dissociation_Beta_vdW', 'Surface_Dissociation_Double',
       'Surface_Dissociation_Double_vdW',
       'Surface_Dissociation_to_Bidentate', 'Surface_Dissociation_vdW',
       'Surface_Monodentate_to_Bidentate', 'Surface_vdW_to_Bidentate', "Surface_Migration",
       "Surface_EleyRideal_Addition_Multiple_Bond" ]
    family = rxn.family
    r = Molecule()
    for react in rxn.reactants:
        r = r.merge(react)
    p = Molecule()
    for prod in rxn.products:
        p = p.merge(prod)
    newrxn = Reaction(reactants=[Species(molecule=[mol]) for mol in rxn.reactants],products=[Species(molecule=[mol]) for mol in rxn.products])
    rstr = str(newrxn)
    
    r.update_multiplicity()
    p.update_multiplicity()
    
    if family not in considered_families:
        raise ValueError
    
    if family == "Surface_Abstraction_Beta_double_vdW":
        #*1--*5 if not exists and *4--*6 if not exists
        for x in [r,p]:
            l1 = x.get_labeled_atoms("*1")[0]
            l5 = x.get_labeled_atoms("*5")[0]
            l4 = x.get_labeled_atoms("*4")[0]
            l6 = x.get_labeled_atoms("*6")[0]
            if not x.has_bond(l1,l5):
                bd = Bond(l1,l5,order="vdW")
                x.add_bond(bd)
            if not x.has_bond(l4,l6):
                bd = Bond(l4,l6,order="vdW")
                x.add_bond(bd)
    elif family == "Surface_Abstraction_Beta_vdW":
        #*1--*5 if not exists
        for x in [r,p]:
            l1 = x.get_labeled_atoms("*1")[0]
            l5 = x.get_labeled_atoms("*5")[0]
            if not x.has_bond(l1,l5):
                bd = Bond(l1,l5,order="vdW")
                x.add_bond(bd)
    elif family == "Surface_Abstraction_Single_vdW":
        #*2--*1 if not exists and *5--*4 if not exists
        for x in [r,p]:
            l1 = x.get_labeled_atoms("*1")[0]
            l2 = x.get_labeled_atoms("*2")[0]
            l4 = x.get_labeled_atoms("*4")[0]
            l5 = x.get_labeled_atoms("*5")[0]
            if not x.has_bond(l1,l2):
                bd = Bond(l1,l2,order="vdW")
                x.add_bond(bd)
            if not x.has_bond(l4,l5):
                bd = Bond(l4,l5,order="vdW")
                x.add_bond(bd)
    elif family == "Surface_Dissociation_Beta_vdW":
        #*1--*4 if not exists
        for x in [r,p]:
            l1 = x.get_labeled_atoms("*1")[0]
            l4 = x.get_labeled_atoms("*4")[0]
            if not x.has_bond(l1,l4):
                bd = Bond(l1,l4,order="vdW")
                x.add_bond(bd)
    elif family == "Surface_Dissociation_Double_vdW":
        #*1--*2 if not exists
        for x in [r,p]:
            l1 = x.get_labeled_atoms("*1")[0]
            l2 = x.get_labeled_atoms("*2")[0]
            if not x.has_bond(l1,l2):
                bd = Bond(l1,l2,order="vdW")
                x.add_bond(bd)
    elif family == "Surface_Dissociation_vdW":
        #*1--*3 if not exists
        for x in [r,p]:
            l1 = x.get_labeled_atoms("*1")[0]
            l3 = x.get_labeled_atoms("*3")[0]
            if not x.has_bond(l1,l3):
                bd = Bond(l1,l3,order="vdW")
                x.add_bond(bd)
    elif family == 'Surface_vdW_to_Bidentate':
        #*1--*3 if not exists
        for x in [r,p]:
            l1 = x.get_labeled_atoms("*1")[0]
            l3 = x.get_labeled_atoms("*3")[0]
            if not x.has_bond(l1,l3):
                bd = Bond(l1,l3,order="vdW")
                x.add_bond(bd)
    elif family == "Surface_Migration":
        #add new empty site *5 and if *3-*1 remove and make *3-*5
        for x in [r,p]:
            newsite = Atom(element="X", lone_pairs=0)
            newsite.label = "*5"
            x.add_atom(newsite)
            l1 = x.get_labeled_atoms("*1")[0]
            l3 = x.get_labeled_atoms("*3")[0]
            if x.has_bond(l1,l3):
                b13 = x.get_bond(l1,l3)
                x.remove_bond(b13)
                bd = Bond(l3,newsite,order="S")
                x.add_bond(bd)
    elif family == "Surface_EleyRideal_Addition_Multiple_Bond":
        #add new empty site *5 and if *3-*1 remove and make *3-*5
        for x in [r,p]:
            newsite = Atom(element="X", lone_pairs=0)
            newsite.label = "*5"
            x.add_atom(newsite)
            l1 = x.get_labeled_atoms("*1")[0]
            l3 = x.get_labeled_atoms("*3")[0]
            if x.has_bond(l1,l3):
                b13 = x.get_bond(l1,l3)
                x.remove_bond(b13)
                bd = Bond(l3,newsite,order="S")
                x.add_bond(bd)
    
    d = dict()
    d["reactant"] = r.to_adjacency_list()
    d["product"] = p.to_adjacency_list()
    d["reaction"] = rstr
    d["reaction_family"] = family
    return d

def get_diffusions(m):
    sitemol = Molecule(atoms=[Atom(element="X", lone_pairs=0)])
    react = Reaction(reactants=[Species(molecule=[m]),Species(molecule=[sitemol])],products=[Species(molecule=[m]),Species(molecule=[sitemol])])
    rstr = str(react)
    mol = m.copy(deep=True)
    siteinds = [mol.atoms.index(x) for x in mol.get_surface_sites()]
    newsite = Atom(element="X", lone_pairs=0)
    newsite.label = "*3"
    mol.add_atom(newsite)
    ds = []
    for siteind in siteinds:
        r = mol.copy(deep=True)
        r.atoms[siteind].label = "*1"
        bdatom = list(r.atoms[siteind].bonds.keys())[0]
        bdatom.label = "*2"
        p = r.copy(deep=True)
        l1 = p.get_labeled_atoms("*1")[0]
        l2 = p.get_labeled_atoms("*2")[0]
        l3 = p.get_labeled_atoms("*3")[0]
        b12 = p.get_bond(l1,l2)
        order = b12.order
        p.remove_bond(b12)
        bd23 = Bond(l2,l3,order=order)
        p.add_bond(bd23)
        d = dict()
        d["reactant"] = r.to_adjacency_list()
        d["product"] = p.to_adjacency_list()
        d["reaction"] = rstr
        d["reaction_family"] = "Diffusion"
        ds.append(d)
    
    return ds

In [48]:
#Give the smiles strings for the reactants and products of the reaction
reactants_products = [
    [["O*","*"],["O=*","[H]*"]],
    [["O=C*","*"],[]], #for empty list products generates all reactions with all products for the given reactants
]


In [49]:
#Generate the labeled reactions
rxns = []
for rp in reactants_products:
    if rp[1]:
        rs = get_labeled_rxns([Molecule().from_smiles(x) for x in rp[0]],[Molecule().from_smiles(x) for x in rp[1]])
    else:
        rs = get_labeled_rxns([Molecule().from_smiles(x) for x in rp[0]],products=None)
    rxns.extend(rs)

In [50]:
rxns

[TemplateReaction(reactants=[Molecule(smiles="[Pt]"), Molecule(smiles="O[Pt]")], products=[Molecule(smiles="O=[Pt]"), Molecule(smiles="[PtH]")], pairs=[[Molecule(smiles="O[Pt]"), Molecule(smiles="O=[Pt]")], [Molecule(smiles="O[Pt]"), Molecule(smiles="[PtH]")]], family='Surface_Dissociation', template=['O-H', 'VacantSite'], comment=None),
 TemplateReaction(reactants=[Molecule(smiles="[Pt]"), Molecule(smiles="O=C[Pt]")], products=[Molecule(smiles="O=C=[Pt]"), Molecule(smiles="[PtH]")], pairs=[[Molecule(smiles="O=C[Pt]"), Molecule(smiles="O=C=[Pt]")], [Molecule(smiles="O=C[Pt]"), Molecule(smiles="[PtH]")]], family='Surface_Dissociation', template=['C-H', 'VacantSite'], comment=None),
 TemplateReaction(reactants=[Molecule(smiles="[Pt]"), Molecule(smiles="O=C[Pt]")], products=[Molecule(smiles="C#[Pt]"), Molecule(smiles="O=[Pt]")], pairs=[[Molecule(smiles="O=C[Pt]"), Molecule(smiles="C#[Pt]")], [Molecule(smiles="O=C[Pt]"), Molecule(smiles="O=[Pt]")]], family='Surface_Dissociation_Double', te

In [51]:
#Filter down to the unique labeled reactions
final_rxns = []
for r in rxns:
    for rf in final_rxns:
        if r.is_isomorphic(rf,either_direction=True,generate_initial_map=True,strict=False):
            break
    else:
        final_rxns.append(r)

In [52]:
len(final_rxns)

4

In [53]:
#Generate Pynta reaction yaml dictionaries
outdicts = [labeled_rxn_to_dict(rxn) for rxn in final_rxns]

In [54]:
outdicts

[{'reactant': '1 *4 X u0 p0 c0\n2 *1 O u0 p2 c0 {3,S} {4,S}\n3 *2 H u0 p0 c0 {2,S}\n4 *3 X u0 p0 c0 {2,S}\n',
  'product': '1 *1 O u0 p2 c0 {2,D}\n2 *3 X u0 p0 c0 {1,D}\n3 *2 H u0 p0 c0 {4,S}\n4 *4 X u0 p0 c0 {3,S}\n',
  'reaction': '[Pt] + O[Pt] <=> O=[Pt] + [PtH]',
  'reaction_family': 'Surface_Dissociation'},
 {'reactant': '1 *4 X u0 p0 c0\n2    O u0 p2 c0 {3,D}\n3 *1 C u0 p0 c0 {2,D} {4,S} {5,S}\n4 *2 H u0 p0 c0 {3,S}\n5 *3 X u0 p0 c0 {3,S}\n',
  'product': '1    O u0 p2 c0 {2,D}\n2 *1 C u0 p0 c0 {1,D} {3,D}\n3 *3 X u0 p0 c0 {2,D}\n4 *2 H u0 p0 c0 {5,S}\n5 *4 X u0 p0 c0 {4,S}\n',
  'reaction': '[Pt] + O=C[Pt] <=> O=C=[Pt] + [PtH]',
  'reaction_family': 'Surface_Dissociation'},
 {'reactant': '1 *4 X u0 p0 c0\n2 *2 O u0 p2 c0 {3,D}\n3 *1 C u0 p0 c0 {2,D} {4,S} {5,S}\n4    H u0 p0 c0 {3,S}\n5 *3 X u0 p0 c0 {3,S}\n',
  'product': '1 *1 C u0 p0 c0 {2,S} {3,T}\n2    H u0 p0 c0 {1,S}\n3 *3 X u0 p0 c0 {1,T}\n4 *2 O u0 p2 c0 {5,D}\n5 *4 X u0 p0 c0 {4,D}\n',
  'reaction': '[Pt] + O=C[Pt] <=>

In [55]:
#Smiles strings for adsorbates diffusion reactions are desired for
diff_adsorbates = ["O*"]

In [56]:
#Generate dictionaries for diffusion reactions
diff_dicts = []
for ad in diff_adsorbates:
    diff_dicts.extend(get_diffusions(Molecule().from_smiles(ad)))

In [57]:
diff_dicts

[{'reactant': '1 *2 O u0 p2 c0 {2,S} {3,S}\n2    H u0 p0 c0 {1,S}\n3 *1 X u0 p0 c0 {1,S}\n4 *3 X u0 p0 c0\n',
  'product': '1 *2 O u0 p2 c0 {2,S} {4,S}\n2    H u0 p0 c0 {1,S}\n3 *1 X u0 p0 c0\n4 *3 X u0 p0 c0 {1,S}\n',
  'reaction': 'O[Pt] + [Pt] <=> O[Pt] + [Pt]',
  'reaction_family': 'Diffusion'}]

In [58]:
#Combine reaction dictionaries
pynta_dicts = outdicts + diff_dicts

In [59]:
#write Pynta yaml input
with open("/Users/mjohns9/test.yaml",'w') as f:
    yaml.dump(pynta_dicts,f)